In [1]:
import os
import json
from pinecone import Pinecone, ServerlessSpec, Index
from sentence_transformers import SentenceTransformer

# Initialize Pinecone
pc = Pinecone(
    api_key='3d87982d-aa12-4905-aa83-976f592e8c1b'
)

index_name = 'coreyapp'

# Check if the index already exists (if not, create it)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  # Dimension should match the embedding model output size
        metric='cosine',  # or any other metric of your choice
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Get the host for the index
index_info = pc.describe_index(index_name)
host = index_info['host']

# Connect to the index
index = Index(name=index_name, host=host, api_key='3d87982d-aa12-4905-aa83-976f592e8c1b')

# Load the embedding model
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

# Load the JSON data
json_file_path = '../SpaCy_Extracted_Data/PINECONE_TAGGED_MASTERDATA_Class.json'
with open(json_file_path, 'r', encoding='utf-8') as f:
    recipes = json.load(f)

# Embedding function
def embed_recipe(ingredients):
    ingredients_str = ' '.join([ingredient['ingredient'] for ingredient in ingredients])
    embedding = model.encode(ingredients_str)
    return embedding

# Prepare and upload data to Pinecone
for recipe in recipes:
    embedding = embed_recipe(recipe['tagged_ingredients'])
    metadata = {
        "name": recipe["name"],
        "url": recipe["url"],
        "ingredients": ', '.join([ingredient['ingredient'] for ingredient in recipe['tagged_ingredients']])
    }
    index.upsert([(recipe['url'], embedding, metadata)])

print("Data uploaded to Pinecone successfully.")


You try to use a model that was created with version 3.0.0.dev0, however, your version is 3.0.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



C:\Users\corey\anaconda3\envs\dev\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Data uploaded to Pinecone successfully.


In [8]:
index_info = pc.describe_index(index_name)
print(index_info)  # Add this line to inspect the structure of the index_info
host = index_info['status']['host']  # Ensure the correct path to get the host

{'dimension': 384,
 'host': 'coreytest-i1yh0gi.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'coreytest',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}


PineconeApiAttributeError: IndexModelStatus has no attribute 'host' at ['['received_data', 'status']']['host']